This code has all algorithm which we proposed and we check it using two types of laplacian that is symmetic and unnormalized laplacian. 

Here we have used edge weigths as (f_i/d_i - f_j/d_j)^2. 

And we are doing it for just for graphs with edge weigth 1 or 0.


In [1]:
import numpy as np
import scipy as sp
from numpy import linalg as LA
import networkx.linalg.algebraicconnectivity as alg
import collections 
import math
import pandas as pd


Results_ncut = pd.DataFrame(index = ['NUP','NP', 'NS', 'UnUP','UnP', 'UnS'])
Results_conductance = pd.DataFrame(index = ['NUP','NP', 'NS', 'UnUP','UnP', 'UnS'])
Graphs = []

In [2]:
# This wil get us the similarity matrix W given the graph and the number of vertices n.
# the vertices are number form 0 to n - 1.
def get_matrix2(graph, n):
    
    w = np.zeros((n, n))
   
    for x in graph.keys() :
        for y in graph[x]:
            w[x][y] = 1
            w[y][x] = 1

    return w
    

In [3]:
def get_unnormalized_laplacian(w):
    
    D = w.sum(axis = 1)
    
    D = np.diag(D)  # The degree matrix D
    
    L = D - w
    
    return L

In [4]:
def get_normalized_laplacian(w):
    
    D = w.sum(axis = 1)
    D_sqrt = np.sqrt(D)
    
    D_1 = np.reciprocal(D_sqrt)
    D_1 = np.diag(D_1)  
    D = np.diag(D)
    L = np.dot(D_1, np.dot(D - w, D_1))
    
    return L

In [5]:
# returns the fielder eigen vector given the similartiy matrix W.

def get_fielder(l, n):  #Similairty matix W

    eigvals, eigvecs = sp.linalg.eigh(l)# eigen values and eigen vectors of the unnormalized laplacian D - W 
    #print("Eigenvalues: ",eigvals, "\n")
    eigvecs = np.round( eigvecs, 5)
    eigvals = np.round(eigvals, 4)
    cardinality = 1
    
    fielder_eigval = eigvals[1]
    
    #print(fielder_eigval)
    while(cardinality < n and abs(eigvals[cardinality] - fielder_eigval)<0.000001):
        cardinality = cardinality + 1
        
    fielder = eigvecs.T[1:cardinality, :] #The 2nd smallest eigen vector of the laplacian L = D - W
    
    return fielder



In [6]:
# Checks weather the graph has to connected components or not i.e is the garph paritioned into two clusters 

def check_partion(w, n): # Similarity matrix W and the number of vetrices n
    
    # We will do bfs and check if the graph is partitioned into two parts or not 
    explored = [0]*n 
    
    queue = collections.deque([0])
    
    queue_size = 1
    
    while(queue_size != 0):
        
        node = queue.popleft()
        queue_size -= 1
        
        if explored[node] == 0:
            
            explored[node] = 1
 
            for i in range(0, n):
                if(w[node][i] != 0):
                    neighbour = i
                    
                    if(explored[neighbour] == 0):
                        queue.append(neighbour)
                        queue_size += 1
    
    # Just need to check if this graph has been partitioned into two parts or not 
    is_partitioned = False
    
    for node in explored:
        if node == 0:
            is_partitioned = True
    
    return is_partitioned  

    


In [7]:
# If the graph has been paritioned into two parts we need to know what these partitions are this function gives us that
# not only does it outputs the clusters but also a vector whose ith index is 1 if its in the first cluster otherwise 0.

def get_partition(w, n): # the similrity matrix W and the number of vertices n
    
    # We will do bfs from a vertex and find out its conected component the other vertices which are not in the
    # connected component are in the other cluster 
    clusters = [[], []]
    explored = [0]*n
    
    
    queue = collections.deque([0])
    
    queue_size = 1
    
    
    while(queue_size != 0):
        
        node = queue.popleft()
        queue_size -= 1
        
        if explored[node] == 0:
            
            explored[node] = 1
 
            for i in range(0, n):
                if(w[node][i] != 0):
                    neighbour = i
                    
                    if(explored[neighbour] == 0):
                        queue.append(neighbour)
                        queue_size += 1
    
    
    #If it a node is connected to vertex zero then its in cluster 0 otherwise its in cluster 1. 
    for node in range(0, n):
        if explored[node] == 1:
            clusters[0].append(node)
        else:
            clusters[1].append(node)
    
    return clusters, explored



In [8]:
def get_edge_weights(f, w, n):
    
    new_weight = []
    d = w.sum(axis = 1)
    
    
    for i in range(0, n):
        for j in range(0, n):
            if(w[i][j] != 0):
                new_weight.append([np.dot((f[:,i]*d[i] - f[:,j]*d[j]).T,f[:,i]*d[i] - f[:,j]*d[j]), i, j])
    
    new_weight = [ [round(i, 3) for i in elem] for elem in new_weight]
    new_weight = sorted(new_weight, reverse= True)
    

    return new_weight

In [9]:
# Here we use our current algorithm to partition the graph
# This returns the clusters, the cluster_name vector and the edeges cut
def get_clusters(f, w, n): # the similarity matrix and the number of vertices
    
    new_weight = get_edge_weights(f, w, n)
    
    new_weight.sort(reverse= True)

    new_weight = collections.deque(new_weight)

    new_w = w.copy() # Here we copy the similarity matrix so that when we make changes to new_w nothing happens to the origianl similarty matrix

    edges_cut = []
    while(check_partion(new_w, n) == False):  # keep on removing edges until we have a parition
        edge_remove = new_weight.popleft()

        u = edge_remove[1]
        v = edge_remove[2]

        if(new_w[u][v] != 0):
            edges_cut.append([u,v])

        new_w[u][v] = 0
        new_w[v][u] = 0
        
    
    clusters, cluster_name = get_partition(new_w, n) 
    
    # this part changes the cluster_name from a list to numpy array (This step helps to write easy codes)
    cluster_name = np.asarray(cluster_name)
    cluster_name = np.reshape(cluster_name, (1,n))
    
    
    return clusters, cluster_name, edges_cut 


In [10]:
def ncut(cluster_name, w, n):
    
    cluster_name = np.asarray(cluster_name)
    cluster_name = np.reshape(cluster_name, (1,n))
    
    mull_1 = cluster_name
    mull_2 = np.ones((1,n)) - mull_1
    
    w_1 = np.dot(mull_1, np.dot(w, np.ones((n,1))))
    w_2 = np.dot(mull_2, np.dot(w, np.ones((n,1))))
    
    
    cut = np.dot(mull_1, np.dot(w,mull_2.T))
    #print(cut, w_1,w_2)
    
    ans = cut*(1/w_1 + 1/w_2)
    ans_1 = cut/min(w_1, w_2)
    
    return ans[0][0], ans_1[0][0] 

    

In [11]:
def get_cut(clusters, w, n):
    cut = []
    for x in clusters[0]:
        for y in clusters[1]:
            if(w[x][y] != 0):
                cut.append([x,y])
    return cut

In [12]:
def get_results(l, n, w):
    
    
    fielder = get_fielder(l, n)
    add_col_ncut = []
    add_col_con = []
    
    print("Fielder Eigenvectors:", fielder)
    print("Edge weights:", get_edge_weights(fielder, w, n))
    
    
    clusters, cluster_predict , edges_cut = get_clusters(fielder, w, n) 
    nc, c = ncut(cluster_predict, w, n)
    print("\nFielder Cardinality:", fielder.shape[0])
    print("\nCurrent Updated Algorithm\n")
    print("Ncut value:", nc)
    print("Conductance:", c)
    print("Clusters:", clusters)
    print("Clusters we get using our current algorithm: ", cluster_predict)
    print("The edges removed by the algorithm:",edges_cut)
    print("The edge cut of the clusters:", get_cut(clusters, w, n))
    
    add_col_ncut.append(nc)
    add_col_con.append(c)
    
    clusters, cluster_predict , edges_cut = get_clusters(np.reshape(fielder[0,:],(1,n)), w, n) 
    nc, c = ncut(cluster_predict, w, n)
    print("\nCurrent Algorithm\n")
    print("Ncut value:", nc)
    print("Conductance:", c)
    print("Clusters:", clusters)
    print("Clusters we get using our current algorithm: ", cluster_predict)
    print("The edges removed by the algorithm:",edges_cut)
    print("The edge-cut of the clusters:", get_cut(clusters, w, n))
    
    add_col_ncut.append(nc)
    add_col_con.append(c)
    
    cluster_predict = [0]*n
    
    clusters = [[], []]
    for i in range(0, n):
        if(fielder[0][i] <= 0):
            cluster_predict[i] = 0
            clusters[0].append(i)
        else:
            cluster_predict[i] = 1
            clusters[1].append(i)
    
    nc, c = ncut(cluster_predict, w, n)
    print("\nSpectral clustering Algorithm\n")
    print("Ncut value:", nc)
    print("Conductance:", c)
    print("Clusters:", clusters)
    print("Clusters we get using our current algorithm: ", cluster_predict)
    print("The edge-cut of the clusters:", get_cut(clusters, w, n))
    add_col_ncut.append(nc)
    add_col_con.append(c)
    
    
    return add_col_ncut, add_col_con

In [13]:
def get_results_1(l, n, w):
    
    
    fielder = get_fielder(l, n)
    add_col_ncut = []
    add_col_con = []
    
    
    clusters, cluster_predict , edges_cut = get_clusters(fielder, w, n) 
    nc, c = ncut(cluster_predict, w, n)
    
    add_col_ncut.append(nc)
    add_col_con.append(c)
    
    clusters, cluster_predict , edges_cut = get_clusters(np.reshape(fielder[0,:],(1,n)), w, n) 
    nc, c = ncut(cluster_predict, w, n)
    
    add_col_ncut.append(nc)
    add_col_con.append(c)
    
    cluster_predict = [0]*n
    
    clusters = [[], []]
    for i in range(0, n):
        if(fielder[0][i] <= 0):
            cluster_predict[i] = 0
            clusters[0].append(i)
        else:
            cluster_predict[i] = 1
            clusters[1].append(i)
    
    nc, c = ncut(cluster_predict, w, n)
    add_col_ncut.append(nc)
    add_col_con.append(c)
    
    
    return add_col_ncut, add_col_con

In [14]:
def unnormalized_results(graph, print_results):

    n = graph[1][0]
    graph = graph[0][0]
    w = get_matrix2(graph, n)

    l = get_unnormalized_laplacian(w)
    
    if(print_results == True):
        print("\nUnnormalized results\n")
        return get_results(l, n, w)
    else:
        return get_results_1(l, n, w)

In [15]:
def normalized_results(graph, print_results):

    n = graph[1][0]
    graph = graph[0][0]
    w = get_matrix2(graph, n)

    l = get_normalized_laplacian(w)
    
    if(print_results == True):
        print("\nNormalized results\n")
        return get_results(l, n, w)
    else:
        return get_results_1(l, n, w)

In [16]:
def show_results(graph, print_results):
    
    add_col_ncut, add_col_con  = [], []
    
    add_col_ncut, add_col_con = normalized_results(graph[1], print_results)
    un, uc = unnormalized_results(graph[1], print_results)
    add_col_ncut += un
    add_col_con += uc
    
    Results_ncut[graph[0]], Results_conductance[graph[0]] = add_col_ncut, add_col_con
    return 0

In [17]:
Petersen = ['Petersen', [[{0 : [2, 3],
         1 : [4, 3],
         2 : [0, 4],
         3 : [0, 1],
         4 : [1, 2],
         5 : [6, 0],
         6 : [7, 1],
         7 : [8, 2],
         8 : [9, 3],
         9 : [5, 4]
             }],
        [10]]]

Graphs.append(Petersen)
show_results(Petersen, True)


Normalized results

Fielder Eigenvectors: [[ 0.      -0.42011 -0.18582 -0.14643  0.04568  0.33225 -0.31935 -0.23149
   0.27368  0.6516 ]
 [ 0.70711 -0.2357   0.2357   0.2357  -0.2357   0.2357  -0.2357  -0.2357
  -0.2357  -0.2357 ]
 [ 0.       0.15314  0.15832 -0.45189  0.42339  0.29357  0.18164 -0.26507
  -0.60503  0.11193]
 [ 0.      -0.45994  0.56507 -0.41371  0.0588  -0.15137 -0.10503  0.50627
   0.04623 -0.04634]
 [ 0.      -0.18154 -0.25602 -0.21828 -0.50955  0.4743   0.54629  0.25353
  -0.03674 -0.07199]]
Edge weights: [[6.0, 9, 8], [6.0, 9, 5], [6.0, 9, 4], [6.0, 8, 9], [6.0, 8, 7], [6.0, 8, 3], [6.0, 7, 8], [6.0, 7, 6], [6.0, 7, 2], [6.0, 6, 7], [6.0, 6, 5], [6.0, 6, 1], [6.0, 5, 9], [6.0, 5, 6], [6.0, 5, 0], [6.0, 4, 9], [6.0, 4, 2], [6.0, 4, 1], [6.0, 3, 8], [6.0, 3, 1], [6.0, 3, 0], [6.0, 2, 7], [6.0, 2, 4], [6.0, 2, 0], [6.0, 1, 6], [6.0, 1, 4], [6.0, 1, 3], [6.0, 0, 5], [6.0, 0, 3], [6.0, 0, 2]]

Fielder Cardinality: 5

Current Updated Algorithm

Ncut value: 1.11111111111

0

In [18]:
Enneahedron = ['Enneahedron', [[{0 : [1, 3, 4],
         1 : [0, 2, 4, 5],
         2 : [1, 3, 6],
         3 : [0, 7, 6, 2],
         4 : [0, 1, 5, 6, 7],
         5 : [1, 4, 6],
         6 : [2, 3, 4, 5, 7],
         7 : [4, 3, 6],
             }],
        [8]]]
Graphs.append(Enneahedron)
show_results(Enneahedron, True)


Normalized results

Fielder Eigenvectors: [[ 0.20265  0.48088 -0.20265 -0.48088  0.28431  0.38322 -0.28431 -0.38322]]
Edge weights: [[8.083, 6, 4], [8.083, 4, 6], [6.611, 7, 4], [6.611, 6, 5], [6.611, 5, 6], [6.611, 4, 7], [6.408, 3, 0], [6.408, 2, 1], [6.408, 1, 2], [6.408, 0, 3], [1.731, 3, 2], [1.731, 2, 3], [1.731, 1, 0], [1.731, 0, 1], [0.662, 6, 2], [0.662, 4, 0], [0.662, 2, 6], [0.662, 0, 4], [0.599, 7, 3], [0.599, 5, 1], [0.599, 3, 7], [0.599, 1, 5], [0.252, 6, 3], [0.252, 4, 1], [0.252, 3, 6], [0.252, 1, 4], [0.074, 7, 6], [0.074, 6, 7], [0.074, 5, 4], [0.074, 4, 5]]

Fielder Cardinality: 1

Current Updated Algorithm

Ncut value: 0.6666666666666666
Conductance: 0.3333333333333333
Clusters: [[0, 1, 4, 5], [2, 3, 6, 7]]
Clusters we get using our current algorithm:  [[1 1 0 0 1 1 0 0]]
The edges removed by the algorithm: [[6, 4], [7, 4], [6, 5], [3, 0], [2, 1]]
The edge cut of the clusters: [[0, 3], [1, 2], [4, 6], [4, 7], [5, 6]]

Current Algorithm

Ncut value: 0.66666666666666

0

In [19]:
Enneahedron_1 = ['Enneahedron_1',[[{0 : [1, 3, 4],
         1 : [0, 2, 4, 6, 5],
         2 : [1, 3, 5],
         3 : [0, 4, 7, 5, 2],
         4 : [0, 1, 3, 6],
         5 : [1, 7, 3, 2],
         6 : [1, 3, 4, 7],
         7 : [6, 5],
             }],
        [8]]]
Graphs.append(Enneahedron_1)
show_results(Enneahedron_1, True)


Normalized results

Fielder Eigenvectors: [[ 0.46236  0.16479 -0.33309 -0.08359  0.50135 -0.49733  0.1138  -0.35984]]
Edge weights: [[7.914, 5, 1], [7.914, 1, 5], [6.285, 4, 3], [6.285, 3, 4], [3.567, 3, 0], [3.567, 0, 3], [3.324, 2, 1], [3.324, 1, 2], [2.403, 6, 4], [2.403, 4, 6], [2.355, 7, 6], [2.355, 6, 7], [2.213, 5, 3], [2.213, 3, 5], [1.396, 4, 1], [1.396, 1, 4], [0.98, 5, 2], [0.98, 2, 5], [0.915, 6, 3], [0.915, 3, 6], [0.828, 7, 5], [0.828, 5, 7], [0.382, 4, 0], [0.382, 0, 4], [0.334, 7, 3], [0.334, 3, 7], [0.317, 1, 0], [0.317, 0, 1], [0.248, 3, 2], [0.248, 2, 3], [0.136, 6, 1], [0.136, 1, 6]]

Fielder Cardinality: 1

Current Updated Algorithm

Ncut value: 0.75
Conductance: 0.375
Clusters: [[0, 1, 4, 6], [2, 3, 5, 7]]
Clusters we get using our current algorithm:  [[1 1 0 0 1 0 1 0]]
The edges removed by the algorithm: [[5, 1], [4, 3], [3, 0], [2, 1], [6, 4], [7, 6], [5, 3], [4, 1], [5, 2], [6, 3]]
The edge cut of the clusters: [[0, 3], [1, 2], [1, 5], [4, 3], [6, 3], [6, 7]]

0

In [20]:
Cockroach = ['Cockroach',[[{0 : [1],
         1 : [0, 2],
         2 : [1, 3],
         3 : [2, 4, 8],
         4 : [3, 5, 7],
         5 : [4, 6],
         6 : [5, 7],
         7 : [6, 8, 4],
         8 : [7, 9, 3],
         9 : [8, 10],
         10 : [11]
             }],
        [12]]]
Graphs.append(Cockroach)
show_results(Cockroach, True)


Normalized results

Fielder Eigenvectors: [[ 0.38235  0.48349  0.32392  0.11732  0.0353   0.01034 -0.01034 -0.0353
  -0.11732 -0.32392 -0.48349 -0.38235]]
Edge weights: [[0.496, 8, 3], [0.496, 3, 8], [0.342, 11, 10], [0.342, 10, 11], [0.342, 1, 0], [0.342, 0, 1], [0.102, 10, 9], [0.102, 9, 10], [0.102, 2, 1], [0.102, 1, 2], [0.088, 9, 8], [0.088, 8, 9], [0.088, 3, 2], [0.088, 2, 3], [0.061, 8, 7], [0.061, 7, 8], [0.061, 4, 3], [0.061, 3, 4], [0.045, 7, 4], [0.045, 4, 7], [0.007, 7, 6], [0.007, 6, 7], [0.007, 5, 4], [0.007, 4, 5], [0.002, 6, 5], [0.002, 5, 6]]

Fielder Cardinality: 1

Current Updated Algorithm

Ncut value: 1.04
Conductance: 1.0
Clusters: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [11]]
Clusters we get using our current algorithm:  [[1 1 1 1 1 1 1 1 1 1 1 0]]
The edges removed by the algorithm: [[8, 3], [11, 10]]
The edge cut of the clusters: [[10, 11]]

Current Algorithm

Ncut value: 1.04
Conductance: 1.0
Clusters: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [11]]
Clusters we get u

0

In [21]:
Cockroach_1 = ['Cockroach_1', [[{0 : [1, 11],
         1 : [0, 2],
         2 : [1, 3],
         3 : [2, 4, 8],
         4 : [3, 5, 7],
         5 : [4, 6],
         6 : [5, 7],
         7 : [6, 8, 4],
         8 : [7, 9, 3],
         9 : [8, 10],
         10 : [11],
         11 : [0]    }],
        [12]]]

Graphs.append(Cockroach_1)
show_results(Cockroach_1, True)


Normalized results

Fielder Eigenvectors: [[-0.40633 -0.30607 -0.13029  0.09509  0.31456  0.34097  0.34097  0.31456
   0.09509 -0.13029 -0.30607 -0.40633]]
Edge weights: [[0.434, 8, 7], [0.434, 7, 8], [0.434, 4, 3], [0.434, 3, 4], [0.298, 9, 8], [0.298, 8, 9], [0.298, 3, 2], [0.298, 2, 3], [0.124, 10, 9], [0.124, 9, 10], [0.124, 2, 1], [0.124, 1, 2], [0.069, 7, 6], [0.069, 6, 7], [0.069, 5, 4], [0.069, 4, 5], [0.04, 11, 10], [0.04, 10, 11], [0.04, 1, 0], [0.04, 0, 1], [0.0, 11, 0], [0.0, 8, 3], [0.0, 7, 4], [0.0, 6, 5], [0.0, 5, 6], [0.0, 4, 7], [0.0, 3, 8], [0.0, 0, 11]]

Fielder Cardinality: 1

Current Updated Algorithm

Ncut value: 0.3111111111111111
Conductance: 0.2
Clusters: [[0, 1, 2, 3, 8, 9, 10, 11], [4, 5, 6, 7]]
Clusters we get using our current algorithm:  [[1 1 1 1 0 0 0 0 1 1 1 1]]
The edges removed by the algorithm: [[8, 7], [4, 3]]
The edge cut of the clusters: [[3, 4], [8, 7]]

Current Algorithm

Ncut value: 0.3111111111111111
Conductance: 0.2
Clusters: [[0, 1, 2, 3, 8

0

In [22]:
Path_point  = ['Path_point',[[{0 : [1],
         1 : [0, 2],
         2 : [1, 3],
         3 : [2, 4],
         4 : [3, 5],
         5 : [4, 6],
         6 : [5, 7],
         7 : [6, 8],
         8: [7, 9],
         9: [8, 10],
         10:[9, 11],
         11 :[0,1,2,3,4,5,6,7,8,9,10]
                }],
        [12]]]

Graphs.append(Path_point)
show_results(Path_point, True)


Normalized results

Fielder Eigenvectors: [[ 0.26648  0.3941   0.38746  0.30771  0.16988  0.      -0.16988 -0.30771
  -0.38746 -0.3941  -0.26648  0.     ]]
Edge weights: [[1.398, 11, 9], [1.398, 11, 1], [1.398, 9, 11], [1.398, 1, 11], [1.351, 11, 8], [1.351, 11, 2], [1.351, 8, 11], [1.351, 2, 11], [0.852, 11, 7], [0.852, 11, 3], [0.852, 7, 11], [0.852, 3, 11], [0.422, 10, 9], [0.422, 9, 10], [0.422, 1, 0], [0.422, 0, 1], [0.284, 11, 10], [0.284, 11, 0], [0.284, 10, 11], [0.284, 0, 11], [0.26, 11, 6], [0.26, 11, 4], [0.26, 6, 11], [0.26, 6, 5], [0.26, 5, 6], [0.26, 5, 4], [0.26, 4, 11], [0.26, 4, 5], [0.171, 7, 6], [0.171, 6, 7], [0.171, 4, 3], [0.171, 3, 4], [0.057, 8, 7], [0.057, 7, 8], [0.057, 3, 2], [0.057, 2, 3], [0.0, 11, 5], [0.0, 9, 8], [0.0, 8, 9], [0.0, 5, 11], [0.0, 2, 1], [0.0, 1, 2]]

Fielder Cardinality: 1

Current Updated Algorithm

Ncut value: 1.05
Conductance: 1.0
Clusters: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11], [10]]
Clusters we get using our current algorithm:  [[1 1 1

0

In [23]:
Two_clusters = ['Two_clusters', [[{0 : [1, 2, 3],
         1 : [0, 2, 3, 4],
         2 : [0, 1, 3],
         3 : [0, 1, 2],
         4 : [5, 6, 7],
         5 : [4, 6, 7],
         6 : [4, 5, 7],
         7 : [4, 5, 6]
        }],
        [8]]]

Graphs.append(Two_clusters)
show_results(Two_clusters, True)


Normalized results

Fielder Eigenvectors: [[-0.37369 -0.28473 -0.37369 -0.37369  0.28473  0.37369  0.37369  0.37369]]
Edge weights: [[5.189, 4, 1], [5.189, 1, 4], [0.0, 7, 6], [0.0, 7, 5], [0.0, 7, 4], [0.0, 6, 7], [0.0, 6, 5], [0.0, 6, 4], [0.0, 5, 7], [0.0, 5, 6], [0.0, 5, 4], [0.0, 4, 7], [0.0, 4, 6], [0.0, 4, 5], [0.0, 3, 2], [0.0, 3, 1], [0.0, 3, 0], [0.0, 2, 3], [0.0, 2, 1], [0.0, 2, 0], [0.0, 1, 3], [0.0, 1, 2], [0.0, 1, 0], [0.0, 0, 3], [0.0, 0, 2], [0.0, 0, 1]]

Fielder Cardinality: 1

Current Updated Algorithm

Ncut value: 0.15384615384615385
Conductance: 0.07692307692307693
Clusters: [[0, 1, 2, 3], [4, 5, 6, 7]]
Clusters we get using our current algorithm:  [[1 1 1 1 0 0 0 0]]
The edges removed by the algorithm: [[4, 1]]
The edge cut of the clusters: [[1, 4]]

Current Algorithm

Ncut value: 0.15384615384615385
Conductance: 0.07692307692307693
Clusters: [[0, 1, 2, 3], [4, 5, 6, 7]]
Clusters we get using our current algorithm:  [[1 1 1 1 0 0 0 0]]
The edges removed by the alg

0

In [24]:
Discrete_moons = {}

for i in range(0, 19):
    Discrete_moons[i] = [i + 1]
    Discrete_moons[i + 20] = [i + 21]

print(Discrete_moons)

for i in range(0, 10):
    Discrete_moons[i].append(19 - i)
    Discrete_moons[i + 20].append(39 - i)

for i in range(0, 5):
    Discrete_moons[i + 10].append(24 - i)
#print(Discrete_moons)

D_M = ['Discrete_moons',[[Discrete_moons], [40]]]

Graphs.append(D_M)
show_results(D_M, True)

{0: [1], 20: [21], 1: [2], 21: [22], 2: [3], 22: [23], 3: [4], 23: [24], 4: [5], 24: [25], 5: [6], 25: [26], 6: [7], 26: [27], 7: [8], 27: [28], 8: [9], 28: [29], 9: [10], 29: [30], 10: [11], 30: [31], 11: [12], 31: [32], 12: [13], 32: [33], 13: [14], 33: [34], 14: [15], 34: [35], 15: [16], 35: [36], 16: [17], 36: [37], 17: [18], 37: [38], 18: [19], 38: [39]}

Normalized results

Fielder Eigenvectors: [[-0.22116 -0.26088 -0.23651 -0.19929 -0.15195 -0.09961 -0.05431 -0.01597
   0.01431  0.02646  0.0493   0.02968 -0.00811 -0.0512  -0.10052 -0.14853
  -0.19836 -0.23626 -0.26081 -0.22114 -0.0493  -0.02968  0.00811  0.0512
   0.10052  0.14853  0.19836  0.23626  0.26081  0.22114  0.22116  0.26088
   0.23651  0.19929  0.15195  0.09961  0.05431  0.01597 -0.01431 -0.02646]]
Edge weights: [[0.116, 31, 30], [0.116, 30, 31], [0.116, 29, 28], [0.116, 28, 29], [0.116, 19, 18], [0.116, 18, 19], [0.116, 1, 0], [0.116, 0, 1], [0.065, 24, 10], [0.065, 20, 14], [0.065, 14, 20], [0.065, 10, 24], [0.039, 2

0

In [25]:
Double_Tree = {}

for i in range(1,10):
    Double_Tree[i] = [math.floor((i - 1)/2)]
    Double_Tree[i + 10] = [10 + math.floor((i - 1)/2)]

Double_Tree[0] = [10]

D_T = ['Double_Tree',[[Double_Tree], [20]]]

Graphs.append(D_T)
show_results(D_T, True)


Normalized results

Fielder Eigenvectors: [[-0.11333 -0.28954 -0.14811 -0.37841 -0.27504 -0.08964 -0.08964 -0.22903
  -0.22903 -0.20388  0.11333  0.28954  0.14811  0.37841  0.27504  0.08964
   0.08964  0.22903  0.22903  0.20388]]
Edge weights: [[0.821, 18, 13], [0.821, 17, 13], [0.821, 13, 18], [0.821, 13, 17], [0.821, 8, 3], [0.821, 7, 3], [0.821, 3, 8], [0.821, 3, 7], [0.462, 10, 0], [0.462, 0, 10], [0.279, 11, 10], [0.279, 10, 11], [0.279, 1, 0], [0.279, 0, 1], [0.126, 16, 12], [0.126, 15, 12], [0.126, 12, 16], [0.126, 12, 15], [0.126, 6, 2], [0.126, 5, 2], [0.126, 2, 6], [0.126, 2, 5], [0.12, 19, 14], [0.12, 14, 19], [0.12, 9, 4], [0.12, 4, 9], [0.101, 14, 11], [0.101, 11, 14], [0.101, 4, 1], [0.101, 1, 4], [0.071, 13, 11], [0.071, 11, 13], [0.071, 3, 1], [0.071, 1, 3], [0.011, 12, 10], [0.011, 10, 12], [0.011, 2, 0], [0.011, 0, 2]]

Fielder Cardinality: 1

Current Updated Algorithm

Ncut value: 1.027027027027027
Conductance: 1.0
Clusters: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1

0

In [26]:
print(D_M)

['Discrete_moons', [[{0: [1, 19], 20: [21, 39], 1: [2, 18], 21: [22, 38], 2: [3, 17], 22: [23, 37], 3: [4, 16], 23: [24, 36], 4: [5, 15], 24: [25, 35], 5: [6, 14], 25: [26, 34], 6: [7, 13], 26: [27, 33], 7: [8, 12], 27: [28, 32], 8: [9, 11], 28: [29, 31], 9: [10, 10], 29: [30, 30], 10: [11, 24], 30: [31], 11: [12, 23], 31: [32], 12: [13, 22], 32: [33], 13: [14, 21], 33: [34], 14: [15, 20], 34: [35], 15: [16], 35: [36], 16: [17], 36: [37], 17: [18], 37: [38], 18: [19], 38: [39]}], [40]]]


In [27]:
Double_Tree1 = {}

for i in range(1,10):
    Double_Tree1[i] = [math.floor((i - 1)/2)]
    Double_Tree1[i + 10] = [10 + math.floor((i - 1)/2)]

Double_Tree1[0] = [10]

for i in range(5,9):
     Double_Tree1[i].append(10+i)
for i in range(0,9):
    Double_Tree1[i].append((i+1)%10)
D_T1 = ['Double_Tree1',[[Double_Tree1], [20]]]

Graphs.append(D_T1)
print(D_T1)
show_results(D_T1, True)



['Double_Tree1', [[{1: [0, 2], 11: [10], 2: [0, 3], 12: [10], 3: [1, 4], 13: [11], 4: [1, 5], 14: [11], 5: [2, 15, 6], 15: [12], 6: [2, 16, 7], 16: [12], 7: [3, 17, 8], 17: [13], 8: [3, 18, 9], 18: [13], 9: [4], 19: [14], 0: [10, 1]}], [20]]]

Normalized results

Fielder Eigenvectors: [[-0.056   -0.15839 -0.17641 -0.16513 -0.17928 -0.17047 -0.14296 -0.07218
  -0.07279 -0.10277  0.12812  0.40892 -0.01959  0.23233  0.57435 -0.07872
  -0.0675   0.07994  0.07969  0.46831]]
Edge weights: [[0.71, 11, 10], [0.71, 10, 11], [0.51, 2, 0], [0.51, 0, 2], [0.463, 19, 14], [0.463, 14, 19], [0.305, 10, 0], [0.305, 0, 10], [0.289, 18, 13], [0.289, 13, 18], [0.288, 17, 13], [0.288, 13, 17], [0.288, 7, 3], [0.288, 3, 7], [0.286, 8, 3], [0.286, 3, 8], [0.281, 13, 11], [0.281, 11, 13], [0.275, 15, 5], [0.275, 5, 15], [0.262, 9, 4], [0.262, 4, 9], [0.217, 1, 0], [0.217, 0, 1], [0.203, 18, 8], [0.203, 8, 18], [0.201, 17, 7], [0.201, 7, 17], [0.196, 12, 10], [0.196, 10, 12], [0.191, 16, 6], [0.191, 6, 16], [

0

In [28]:
for graph in Graphs:
    show_results(graph, False)
    
    

In [29]:
print(Results_conductance.T)


                     NUP        NP        NS      UnUP       UnP       UnS
Petersen        1.000000  1.000000  0.500000  1.000000  0.666667  0.466667
Enneahedron     0.333333  0.333333  0.333333  0.333333  0.333333  0.333333
Enneahedron_1   0.375000  0.375000  0.375000  0.714286  0.714286  0.375000
Cockroach       1.000000  1.000000  0.230769  1.000000  1.000000  0.230769
Cockroach_1     0.200000  0.200000  0.166667  0.200000  0.200000  0.166667
Path_point      1.000000  1.000000  0.428571  0.428571  0.428571  0.428571
Two_clusters    0.076923  0.076923  0.076923  0.076923  0.076923  0.076923
Discrete_moons  0.500000  0.500000  0.081967  0.500000  0.500000  0.081967
Double_Tree     1.000000  1.000000  0.052632  1.000000  1.000000  0.052632
Double_Tree1    1.000000  1.000000  0.250000  0.333333  0.333333  0.333333


In [30]:
print(Results_ncut.T)

                     NUP        NP        NS      UnUP       UnP       UnS
Petersen        1.111111  1.111111  0.833333  1.111111  0.833333  0.933333
Enneahedron     0.666667  0.666667  0.666667  0.666667  0.666667  0.666667
Enneahedron_1   0.750000  0.750000  0.750000  0.914286  0.914286  0.750000
Cockroach       1.040000  1.040000  0.461538  1.040000  1.040000  0.461538
Cockroach_1     0.311111  0.311111  0.291667  0.311111  0.311111  0.291667
Path_point      1.050000  1.050000  0.642857  0.642857  0.642857  0.642857
Two_clusters    0.153846  0.153846  0.153846  0.153846  0.153846  0.153846
Discrete_moons  0.516949  0.516949  0.163934  0.516949  0.516949  0.163934
Double_Tree     1.027027  1.027027  0.105263  1.027027  1.027027  0.105263
Double_Tree1    1.016393  1.016393  0.336957  0.369048  0.369048  0.413333


In [31]:
print(Results_conductance.loc['UnP', :])

Petersen          0.666667
Enneahedron       0.333333
Enneahedron_1     0.714286
Cockroach         1.000000
Cockroach_1       0.200000
Path_point        0.428571
Two_clusters      0.076923
Discrete_moons    0.500000
Double_Tree       1.000000
Double_Tree1      0.333333
Name: UnP, dtype: float64


In [32]:
optimal_cuts = []

for i in range(1, 2**10 - 1):
    cluster_predict = [1]*10
    
    for j in range(0,10):
        if(2**j & i != 0):
            cluster_predict[j] = 0
    
    #print(cluster_predict)
    graph = Petersen[1][0]
    graph = graph[0]
    n = Petersen[1][1]
    n  = n[0]
    #print(ncut(cluster_predict, get_matrix2(graph, n), n))
    if(ncut(cluster_predict, get_matrix2(graph, n), n) == ncut([1,1,1,1,1,0,0,0,0,0], get_matrix2(graph, n), n)):
        
        clusters = [[], []]
        for i in range(0, 10):
            clusters[cluster_predict[i]].append(i)
            
        optimal_cuts.append(clusters)

for x in optimal_cuts:
    print(x)
print(len(optimal_cuts))

[[0, 1, 2, 3, 4], [5, 6, 7, 8, 9]]
[[0, 1, 3, 5, 6], [2, 4, 7, 8, 9]]
[[1, 2, 4, 6, 7], [0, 3, 5, 8, 9]]
[[0, 2, 5, 6, 7], [1, 3, 4, 8, 9]]
[[0, 2, 3, 7, 8], [1, 4, 5, 6, 9]]
[[1, 3, 6, 7, 8], [0, 2, 4, 5, 9]]
[[0, 2, 4, 5, 9], [1, 3, 6, 7, 8]]
[[1, 4, 5, 6, 9], [0, 2, 3, 7, 8]]
[[1, 3, 4, 8, 9], [0, 2, 5, 6, 7]]
[[0, 3, 5, 8, 9], [1, 2, 4, 6, 7]]
[[2, 4, 7, 8, 9], [0, 1, 3, 5, 6]]
[[5, 6, 7, 8, 9], [0, 1, 2, 3, 4]]
12
